In [1]:
import pandas as pd
from tqdm import tqdm
import nltk

In [2]:
# nltk.download('punkt')
# nltk.download('stopwords')

In [3]:
acordaos = pd.read_csv('validate-400-2500.csv', encoding='utf-8', error_bad_lines=False, engine="python")

In [4]:
acordaos.head()

,ementa,inteiro_teor
0,agravo de instrumento em recurso de revista – ...,a c ó r d ã o\n\n(ac. (8ª turma) gmmea/mtr/mas...
1,embargos de declaração. acolhimento para prest...,"a c ó r d ã o\n\n(ac. sdi-1) bp/lc \n\nvistos,..."
2,agravo de instrumento em recurso de revista – ...,acórdão\n\n1ª turma vmf/rej/pcp/wmc \n\nvistos...
3,agravo de instrumento. recurso de revista. emb...,"acórdão\n\n4ª turma gmfeo/gs/cjj \n\nvistos, r..."
4,agravo de instrumento.\n\nnega-se provimento a...,a c ó r d ã o\n\n(ac. (5ª turma) bp/cr \n\nvis...


In [5]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest

In [6]:
stopwords = set(stopwords.words('portuguese') + list(punctuation))

In [7]:
def get_text_summary(text, phrases_quantity = 4):
    phrases = sent_tokenize(text)
    words = word_tokenize(text.lower())
    valid_words = [word for word in words if word not in stopwords]
    freq = FreqDist(valid_words)
    
    important_phrases = defaultdict(int)
    
    for i, phrase in enumerate(phrases):
        for word in word_tokenize(phrase.lower()):
            if word in freq:
                important_phrases[i] += freq[word]
                
    important_phrases_indexes = nlargest(phrases_quantity, important_phrases, important_phrases.get)
    
    summary = []
    for i in sorted(important_phrases_indexes):
        summary.append(phrases[i])
        
    return ''.join(summary)

In [8]:
def generate_summaries(texts):
    summaries = []
    for text in tqdm(texts):
        summaries.append(get_text_summary(text, 3))
    
    return summaries

In [ ]:
%%time
generate_summaries(acordaos['inteiro_teor'].tolist())

In [12]:
acordaos_summarized = pd.DataFrame({ 'inteiro_teor': acordaos['inteiro_teor'].tolist(), 'ementa_original': acordaos['ementa'].tolist(), 'resumo_gerado': generate_summaries(acordaos['inteiro_teor'].tolist()) })
acordaos_summarized.to_csv('predictions.csv')

100%|██████████| 2698/2698 [00:17<00:00, 156.12it/s]


In [17]:
generated_summaries = pd.read_csv('predictions.csv', encoding='utf-8', error_bad_lines=False, engine="python")
for index, row in generated_summaries[10:30].iterrows():
    print(f'Exemplo {index}', '\n')
    print('Ementa original:', row['ementa_original'].replace('\n\n', ' '), '\n\n')
    print('Sumário gerado:', row['resumo_gerado'].replace('\n\n', ' '), '\n\n')

Exemplo 10 

Ementa original: agravo de instrumento em recurso de revista - procedimento sumaríssimo. enquadramento sindical. art. 896, § 6º, da clt. nas causas sujeitas ao procedimento sumaríssimo, somente será admitido recurso de revista por contrariedade a súmula de jurisprudência uniforme do tribunal superior do trabalho e violação direta da constituição da república. agravo de instrumento a que se nega provimento. 


Sumário gerado: (8ª turma) gmmea/hagb/acnv  vistos, relatados e discutidos estes autos de agravo de instrumento em recurso de revista n° tst-airr-127740-66.2008.5.02.000, em que é agravante joyce de lima e são agravadas silva e silva advogados associados e cooperativa de trabalho dos profissionais de crédito e cobrança - cccoop.a agravante, reiterando as razões de seu recurso de revista, sustenta ser beneficiária da convenção coletiva dos operadores de telemarketing, vez que exercia na reclamada atividades congêneres.896, § 6º, da clt, tratando-se de causa sujeita ao 

In [19]:
# !pip install rouge-score tqdm --user
from tqdm import tqdm_notebook
# import time
from rouge_score import rouge_scorer, scoring
from typing import List, Dict

In [20]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [21]:
rouge_keys = ["rouge1", "rouge2", "rougeL"]
def calculate_rouge(output_lns: List[str], reference_lns: List[str]) -> Dict:
    scorer = rouge_scorer.RougeScorer(rouge_keys, use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for reference_ln, output_ln in tqdm_notebook(zip(reference_lns, output_lns)):
        scores = scorer.score(reference_ln, output_ln)
        aggregator.add_scores(scores)

    result = aggregator.aggregate()
    return {k: v.mid.fmeasure for k, v in result.items()}

In [24]:
generated_summaries = generated_summaries.drop(['Unnamed: 0'], axis=1)
generated_summaries = generated_summaries.applymap(lambda x: x.replace('\n\n', ' '))

In [26]:
metrics = calculate_rouge(generated_summaries['ementa_original'], generated_summaries['resumo_gerado'])

In [27]:
metrics

{'rouge1': 0.3307905051657543,
 'rouge2': 0.16685518365890264,
 'rougeL': 0.23905559318934727}